In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision import transforms
from torchvision.utils import save_image
import torch.optim as optim

# torch.cuda=torch.cuda.empty_cache()
# import gc
# del variables
# gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

C:\Users\ASUS\anaconda3\envs\saba\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\ASUS\anaconda3\envs\saba\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


Files already downloaded and verified
Files already downloaded and verified


In [10]:
import torch.nn as nn


class AlexNet(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.6),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )

    def forward(self, x):
        conv_features = self.features(x)
        flatten = conv_features.view(conv_features.size(0), -1)
        fc = self.fc_layers(flatten)
        return fc

In [11]:
import torch.optim as optim

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# # AlexNet.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(AlexNet.parameters(), lr=0.001, momentum=0.9)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AlexNet() 
model = model.to(device=device) 

## Loss and optimizer
learning_rate = 1e-4 
load_model = True
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate) 


import time
for epoch in range(7):  

    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
#         scores = model(data)
#         loss = criterion(scores,targets)
#         loss.backward()
#         optimizer.step()
#         loss_ep += loss.item()
        
        # forward + backward + optimize
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        #Time
        end_time = time.time()
        time_taken = end_time - start_time

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            print('Time:',time_taken)
            running_loss = 0.0

print('Finished Training of AlexNet')

[1,  2000] loss: 1.921
Time: 109.79388880729675
[1,  4000] loss: 1.576
Time: 213.33439755439758
[1,  6000] loss: 1.419
Time: 319.1564643383026
[1,  8000] loss: 1.302
Time: 424.67108845710754
[1, 10000] loss: 1.209
Time: 531.2950711250305
[1, 12000] loss: 1.114
Time: 638.0463304519653
[2,  2000] loss: 1.007
Time: 114.68207263946533
[2,  4000] loss: 0.969
Time: 219.15903544425964
[2,  6000] loss: 0.948
Time: 323.0640630722046
[2,  8000] loss: 0.931
Time: 426.4295208454132
[2, 10000] loss: 0.877
Time: 530.8686780929565
[2, 12000] loss: 0.857
Time: 635.7542481422424
[3,  2000] loss: 0.747
Time: 112.25724291801453
[3,  4000] loss: 0.747
Time: 218.93959259986877
[3,  6000] loss: 0.710
Time: 325.68727254867554
[3,  8000] loss: 0.716
Time: 432.0996022224426
[3, 10000] loss: 0.715
Time: 539.1009304523468
[3, 12000] loss: 0.703
Time: 644.1996545791626
[4,  2000] loss: 0.584
Time: 112.11592888832092
[4,  4000] loss: 0.576
Time: 219.06439685821533
[4,  6000] loss: 0.579
Time: 324.4422824382782
[4,

In [19]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %.2f %%' % (100 * correct / total))

Accuracy of the network on the test images: 79.31 %


In [17]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 79 %
Accuracy of   car : 91 %
Accuracy of  bird : 68 %
Accuracy of   cat : 63 %
Accuracy of  deer : 75 %
Accuracy of   dog : 71 %
Accuracy of  frog : 83 %
Accuracy of horse : 82 %
Accuracy of  ship : 92 %
Accuracy of truck : 86 %


In [18]:
#Verifying average accuracy of the network
avg = 0
for i in range(10):
  temp = (100 * class_correct[i] / class_total[i])
  avg = avg + temp
avg = avg/10
print('Average accuracy = ', avg)  

Average accuracy =  79.34
